<h2>Lab 3: Least Squares</h2>

<b>Demo Date: </b> Oct. 13 <br>
<b>Due Date: </b> Oct. 15

In this lab you will implement two algorithms for solving an ill-conditioned least squares problem. We create an artificial and overdetermined least-squares problem by removing two columns of a $10 \times 10$ Hilbert matrix, a classic ill-conditioned matrix. As we remove the two columns from the matrix it is no longer a Hilbert matrix, but it creates an overdetermined system with a large condition number: $\approx 3,796,554,168$.

Implement an algorithm that you believe will compute the value of $x$ for the least squares problem $Ax \approx b$ as accurately as the problem allows. You will also implement an algorithm that you believe to produce inaccurate solutions due to the large condition number of the system. In this lab you do not need to worry about having a fast implementation, any non-vectorized implementation will be enough. 

Compare the two solutions in terms of the norm of their residual vector $Ax - b$. In the demo of your lab you should be able to explain your choices of algorithms and the results you obtained. Why did one method produce a solution with a smaller residual than the other? 

In [ ]:
import numpy as np
import copy
import random
from scipy.linalg import hilbert
from scipy.optimize import lsq_linear

b = np.ones(10)
A = hilbert(10)
A = np.delete(A, 5, 1)
A = np.delete(A, 4, 1)

print('Condition number of A: ', np.linalg.norm(A) * np.linalg.norm(np.linalg.pinv(A)))


b_1 = np.ones(10)
A_1 = hilbert(10)
A_1 = np.delete(A, 5, 1)
A_1 = np.delete(A, 4, 1)

print('Condition number of A1: ', np.linalg.norm(A_1) * np.linalg.norm(np.linalg.pinv(A_1)))

# Use this test instance while developing the algorithms. 
# This instance is much easier to debug than the ill-conditioned problem above. 
A_test = np.array([[1, 2, 2], [4, 4, 2], [4, 6, 4]], dtype=float)
A_test2 = np.array([[1, 2, 2], [4, 4, 2], [4, 6, 4]], dtype=float)
b_test = np.array([3, 6, 10], dtype=float)
b_test2 = np.array([3, 6, 10], dtype=float)

Condition number of A:  3796554168.541331
Condition number of A1:  51447888.49081164


In [ ]:
 def back_substituion(U, b):
  n= len(U)
  x=np.zeros(n)
  for j in range(n-1,-1,-1):
    if U[j][j]==0:
      break
    x[j] = b[j]/U[j][j]
    for i in range(j):
      b[i] = b[i] - U[i][j]*x[j]
  return x

def forward_substituion(L, b):
  n=len(L)
  y=np.zeros(n)
  for j in range(n):
    if L[j][j]==0:
      break
    y[j] = b[j]/L[j][j]
    for i in range(j+1,n):
      b[i] = b[i] - L[i][j]*y[j]
  return y    

def calculate(Q,b):
  Q_T = np.transpose(Q)
  C = np.dot(Q_T,b)
  #print("C is:",C)
  return C

In [ ]:
def accurate(A): #MGS
  n = A.shape[1]
  Q = np.zeros((A.shape[0],A.shape[0])) #10*10
  R = np.zeros((n,n))  #8*8

  for i in range(0,n):  #loop over column
    R[i][i] = np.linalg.norm(A[:,i])
    if R[i][i] == 0:  #stop if linearly dependent
      break
    else:
      Q[:,i] = A[:,i]/R[i][i]  #normalize current column
      for j in range(i+1,n):  #subtract from succeeding columns them components in current column
        q_T = np.transpose(Q[:,i])
        R[i][j] = np.dot(q_T,A[:,j])
        A[:,j] -= R[i][j]*Q[:,i]

#  print("Q is:",Q)
#  print("R is:",R)
  return Q,R

A1 = A
b1 = b
Q_a,R_a = accurate(A1)
C_a = calculate(Q_a,b1)
x_a = back_substituion(R_a.T,C_a)
residual = np.linalg.norm(np.dot(A1,x_a) - b1)
print("residual is:",residual)

residual is: 1.3094887165295381e-05


In [ ]:
def inaccurate(A): #CGS
  n = A.shape[1]
  Q = np.zeros((A.shape[0],A.shape[0])) #10*10
  R = np.zeros((n,n))  #8*8

  for i in range(0,n):  #loop over columns
    Q[:,i] = A[:,i]
    for j in range(0,i):  #subtract from current column its components in preceding columns
      Q_T = np.transpose(Q[:,j])
      R[j][i] = np.dot(Q_T,A[:,i])
      Q[:,i] -= R[j][i]*Q[:,j]
    R[i][i] = np.linalg.norm(Q[:,i])
    if R[i][i] == 0:  #stop if linear dependent
      break
    Q[:,i] = Q[:,i]/R[i][i]  #normalize current column
  
  return Q,R

A2 = A_1
b2 = b_1
Q_i,R_i =inaccurate(A2)
C_i = calculate(Q_i,b2)
x_i = back_substituion(R_i,C_i)
residual = np.linalg.norm(np.dot(A2,x_i) - b2)
print("residual is:",residual)



residual is: 0.0002685582571750269
